In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from selenium.common.exceptions import NoSuchElementException
import re
import time

In [3]:
import pandas as pd
import numpy as np

In [4]:
import requests
import random
from bs4 import BeautifulSoup as bs
import traceback

In [5]:
opts = webdriver.ChromeOptions()
opts.headless =True

In [7]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(10)

In [8]:
search_url = "https://www.proquest.com/washingtonpost?accountid=29989"
driver.get(search_url)


In [9]:
driver.find_element(By.XPATH, '//*[@id="product-level-nav"]/li[2]').click()

In [10]:
keys = """((federal reserve) OR (fed)) AND((money supply) OR (open market operations) OR
(quantitative easing) OR (monetary policy) OR (fed funds rate) OR (overnight lending rate) OR
Bernanke OR Volker OR Greenspan OR Yellen OR Powell OR (central bank) OR (interest rates) OR (fed chairman) OR
(fed chair) OR (lender of last resort) OR (discount window) OR (European Central Bank) OR ECB OR (Bank of England) OR
(Bank of Japan) OR BOJ OR (Bank of China) OR Bundesbank OR (Bank of France) OR (Bank of Italy))"""

In [11]:
driver.find_element(By.XPATH,'//*[@id="select_multiDateRange"]/option[10]').click()
# Start
driver.find_element(By.XPATH,'//*[@id="month2"]/option[2]').click()
driver.find_element(By.XPATH,'//*[@id="day2"]/option[2]').click()
driver.find_element(By.XPATH,'//*[@id="year2"]').send_keys('2000')
# End
driver.find_element(By.XPATH,'//*[@id="month2_0"]/option[13]').click()
driver.find_element(By.XPATH,'//*[@id="day2_0"]/option[32]').click()
driver.find_element(By.XPATH,'//*[@id="year2_0"]').send_keys('2021')

In [12]:
driver.find_element(By.XPATH,'//*[@id="SourceType_Newspapers"]').click()
driver.find_element(By.XPATH,'//*[@id="Language_ENG"]').click()

In [13]:
driver.find_element(By.XPATH, '//*[@id="queryTermField"]').send_keys(keys)

In [14]:
driver.find_element(By.XPATH, '//*[@id="sortType"]/option[2]').click() # oldest first

In [15]:
#driver.implicitly_wait(10)
#driver.find_element(By.XPATH, '//*[@id="itemsPerPage"]/option[4]').click()

In [16]:
dataset = []
pattern_date = '[0-9]{1,2}\s[A-Za-z]{3,5}\s[0-9]{4}'

In [ ]:
num_pages_visited = 0

while True:
    
    articles = driver.find_elements(By.CLASS_NAME, 'resultItem')
    num_articles_scraped = 0 # per page
    for article in articles:
        title = article.find_element(By.XPATH, './/*[@id="citationDocTitleLink"]').get_attribute("title")
        try:
            date_text = article.find_element(By.CLASS_NAME,'newspaperArticle').text
            date = re.search(pattern_date, date_text).group()
        except:
            date = np.nan
    
        link = article.find_element(By.XPATH, './/*[@id="citationDocTitleLink"]').get_attribute("href")
        
        driver_temp = webdriver.Chrome(options = opts)
        driver_temp.implicitly_wait(10)
        driver_temp.get(link)
    
        try:
            full_text = driver_temp.find_element(By.XPATH, '//*[@id="fullTextZone"]/text').text
        except NoSuchElementException:
            full_text = np.nan

        try:
            abstract = driver_temp.find_element(By.CLASS_NAME,'abstract').text
        except NoSuchElementException:
            abstract = np.nan
        
    
        article_info = {'date': date, 'title': title, "abstract": abstract, 'full_text': full_text}
        dataset.append(article_info)
        
        num_articles_scraped += 1
        print(f"Number of scraped articles per page: {num_articles_scraped}")
        driver_temp.close()
        time.sleep(random.uniform(5, 15))
    
    print("\n")
    try:
        num_pages_visited += 1
        print(f"Number of scraped pages: {num_pages_visited}\n")
        driver.find_element(By.CSS_SELECTOR, "[title^='Next Page']").click()
        time.sleep(random.uniform(10, 20))
    except NoSuchElementException:
        break
        
driver.quit()
df = pd.DataFrame(dataset)

In [18]:
df = pd.DataFrame(dataset)

In [ ]:
df

In [20]:
#df.to_csv("news_data.csv", index = False)